<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/dscnn_speech_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#DEFAULT WORD SET
#%env WORDS=yes,no,up,down,left,right,on,off,stop,go

#ALTERNATE WORD SETS
%env WORDS=yes,no,up,down,left,right,on,off,stop,four

In [0]:
!rm -r work

In [0]:
!ls -l work/DS_CNN/DS_CNN1/training/best

In [0]:
!rm -r ML-KWS-for-MCU

In [0]:
!git clone https://github.com/villasen/ML-KWS-for-MCU.git

Train model

In [0]:
!python ML-KWS-for-MCU/train.py \
--data_dir work/speech_dataset/ \
--summaries_dir work/DS_CNN/DS_CNN1/retrain_logs \
--train_dir work/DS_CNN/DS_CNN1/training \
--wanted_words $WORDS

Test model --   Need to update the checkpoint file if training different networks.

In [0]:
!python ML-KWS-for-MCU/test.py \
--data_dir work/speech_dataset/ \
--checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_8805.ckpt-2800 \
--wanted_words $WORDS

Freeze graph to protobuf file

In [0]:
!python ML-KWS-for-MCU/freeze.py \
--summaries_dir work/DS_CNN/DS_CNN1/retrain_logs \
--train_dir work/DS_CNN/DS_CNN1/training \
--checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_9543.ckpt-10400 \
--output_file my_dscnn.pb \
--wanted_words $WORDS

Test protobuf file

In [0]:
!python ML-KWS-for-MCU/label_wav.py \
--wav 0c09f202_nohash_0.wav \
--graph /content/my_dscnn.pb \
--labels /content/work/DS_CNN/DS_CNN1/training/ds_cnn_labels.txt \
--how_many_labels 10

Fold Batchnorm (needed for quantization of ds-cnn)

In [0]:
!python ML-KWS-for-MCU/fold_batchnorm.py \
--data_dir work/speech_dataset/ \
--checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_9543.ckpt-10400 \
--wanted_words $WORDS

Quantize model to weights.h file

In [0]:
!python ML-KWS-for-MCU/quant_test.py \
--data_dir work/speech_dataset/ \
--checkpoint work/DS_CNN/DS_CNN1/training/best/ds_cnn_9543.ckpt-10400_bnfused \
--wanted_words $WORDS

Download weights.h

In [0]:
from google.colab import files

files.download('weights.h')

Download model as tar file

In [0]:
!tar -cvf dscnn__defaultDataset_4000.tar /content/work/
!ls -l

from google.colab import files

files.download('dscnn__defaultDataset_4000.tar')

In [0]:
!rm -r ESC-50


In [0]:
import os
import shutil
import urllib
import zipfile
import glob

def download_dataset(name):
    """Download the dataset into current working directory."""
    if not os.path.exists(name):
        os.mkdir(name)
        urllib.urlretrieve('https://github.com/karoldvl/{0}/archive/master.zip'.format(name), '{0}/{0}.zip'.format(name))

        with zipfile.ZipFile('{0}/{0}.zip'.format(name)) as package:
            package.extractall('{0}/'.format(name))

        os.unlink('{0}/{0}.zip'.format(name))        

        for src in glob.glob('{0}/{0}-master/*'.format(name)):
            shutil.move(src, '{0}/{1}'.format(name, os.path.basename(src)))

        os.rmdir('{0}/{0}-master'.format(name))
        #os.rmdir('ESC-50-master'.format(name))
        
#download_dataset('ESC-10')
download_dataset('ESC-50')

In [0]:
!ls -l ESC-50
